In [1]:
import random
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set, Tuple

import networkx as nx
import numpy as np
import pandas as pd

from icm_diffusion import simulate_diffusion_ICM
from src.graph import (
    create_polarized_graph,
    graph_loader,
    random_color_graph,
    spectral_bipartition_coloring,
)
from src.seed import seed_degree, seed_random


In [2]:
from edge_addition import (
    edge_addition_custom,
    edge_addition_adamic_adar,
    edge_addition_preferential_attachment,
    edge_addition_jaccard,
    edge_addition_degree,
    edge_addition_topk,
    edge_addition_prob,
    edge_addition_kkt,
    edge_addition_random,
)

In [3]:
# Function to evaluate and compare the graph modifications
def evaluate_graph_modifications(G, seeds, k, max_iter):
    # Simulate diffusion on the original graph
    count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seeds, 1, max_iter)
    
    # Results for the original graph
    original_results = pd.DataFrame({
        'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
        'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
    })

    # Define a list of modification functions
    modification_functions = {
        'Adamic Adar': edge_addition_adamic_adar,
        'PrefAtt': edge_addition_preferential_attachment,
        'Jaccard': edge_addition_jaccard,
        'Degree': edge_addition_degree,
        'TopK': edge_addition_topk,
        'Prob': edge_addition_prob,
        'KKT': edge_addition_kkt,
        'Random': edge_addition_random,
        'Custom': edge_addition_custom
    }

    combined_results = original_results.copy()

    # Evaluate each graph modification
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        count, count_std, color_count, color_count_std = simulate_diffusion_ICM(modified_graph, seeds, 1, max_iter)

        adapted_results = pd.DataFrame({
            'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
            f'Adapted Graph {method_name}': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
        })

        combined_results = pd.merge(combined_results, adapted_results, on='Metric')

    # Get the number of nodes and edges for all graphs
    graph_info = {
        'Metric': ['Number of Nodes', 'Number of Edges'],
        'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    }
    
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        graph_info[f'Adapted Graph {method_name}'] = [modified_graph.number_of_nodes(), modified_graph.number_of_edges()]
    
    graph_info_df = pd.DataFrame(graph_info)

    # Combine all results into one DataFrame
    final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

    # Transpose the DataFrame and set the first row as the header
    final_results = final_results.T
    final_results.columns = final_results.iloc[0]  # Set the first row as the column names
    final_results = final_results.drop(final_results.index[0])  # Drop the first row

    return final_results

In [4]:
G = create_polarized_graph(1000, 0.2, 0.01)
#G = graph_loader('datasets/congress_network/congress.edgelist')
#color the graph
spectral_bipartition_coloring(G)

/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [5]:
seed = seed_degree(G, 100)
k = 15
max_iter = 1000
final_results = evaluate_graph_modifications(G, seed, k, max_iter)
final_results

100%|██████████| 1000/1000 [00:10<00:00, 95.58it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,1000.0,102716.0,254.781,147.795,7.64,11.975
Adapted Graph Adamic Adar,1000.0,103907.0,257.707,141.26,7.214,10.817
Adapted Graph PrefAtt,1000.0,104216.0,266.22,149.459,14.037,20.108
Adapted Graph Jaccard,1000.0,103922.0,253.405,140.937,6.895,11.054
Adapted Graph Degree,1000.0,104021.0,247.337,144.952,7.769,11.632
Adapted Graph TopK,1000.0,104042.0,260.979,149.142,9.742,13.687
Adapted Graph Prob,1000.0,104216.0,264.885,144.3,15.107,21.029
Adapted Graph KKT,1000.0,104005.0,254.007,146.645,7.24,12.102
Adapted Graph Random,1000.0,104053.0,253.161,146.772,10.338,17.65
Adapted Graph Custom,1000.0,104056.0,247.418,141.615,10.946,16.321


In [6]:
seed = seed_random(G, 100)
k = 15
max_iter = 1000
final_results = evaluate_graph_modifications(G, seed, k, max_iter)
final_results

100%|██████████| 1000/1000 [00:10<00:00, 98.41it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,1000.0,102716.0,247.298,145.272,6.076,9.289
Adapted Graph Adamic Adar,1000.0,103962.0,253.893,146.627,6.34,9.998
Adapted Graph PrefAtt,1000.0,104216.0,266.602,145.66,13.575,18.016
Adapted Graph Jaccard,1000.0,103957.0,262.838,155.958,6.732,10.671
Adapted Graph Degree,1000.0,104084.0,237.866,142.044,8.028,11.84
Adapted Graph TopK,1000.0,104047.0,243.114,137.762,8.997,13.079
Adapted Graph Prob,1000.0,104216.0,256.498,147.497,13.213,17.706
Adapted Graph KKT,1000.0,104052.0,247.225,148.384,9.958,15.438
Adapted Graph Random,1000.0,104085.0,247.164,145.642,10.047,15.356
Adapted Graph Custom,1000.0,104080.0,250.42,145.277,10.394,16.498
